In [7]:
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.utils import load_img, img_to_array
import wget
import zipfile
import os

In [3]:
# using their dataset
train = wget.download("https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip", "data/")

# using their validation data
val = wget.download("https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip", "data/")

In [4]:
print("Train:", train)
print("Val:", val)

Train: data//horse-or-human.zip
Val: data//validation-horse-or-human.zip


In [5]:
# Unzip training set
local_zip = 'data/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('data/horse-or-human')

# Unzip validation set
local_zip = 'data/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('data/validation-horse-or-human')

zip_ref.close()

In [24]:
train_dir = os.path.join("data/horse-or-human/")
val_dir = os.path.join("data/validation-horse-or-human/")

In [25]:
# create datagenerators for train and validation datasets
generator = ImageDataGenerator(rescale=1./255)

train_datagen = generator.flow_from_directory(
    directory=train_dir,
    target_size=(256,256),
    batch_size=128,
    class_mode="binary"
)

val_datagen = generator.flow_from_directory(
    directory=val_dir,
    target_size=(256,256),
    batch_size=32,
    class_mode="binary"
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [26]:
# building out the model with 4 conv2d + maxpool layers, and a final relu + sigmoid layer
model = keras.Sequential([
    keras.layers.Conv2D(16, (3,3), activation="relu", input_shape=(256,256,3)),
    keras.layers.MaxPool2D(2,2),
    keras.layers.Conv2D(32, (3,3), activation="relu"),
    keras.layers.MaxPool2D(2,2),
    keras.layers.Conv2D(64, (3,3), activation="relu"),
    keras.layers.MaxPool2D(2,2),
    keras.layers.Conv2D(64, (3,3), activation="relu"),
    keras.layers.MaxPool2D(2,2),
    keras.layers.Conv2D(64, (3,3), activation="relu"),
    keras.layers.MaxPool2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

# view summary of model
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 127, 127, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 62, 62, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 30, 30, 64)      

In [27]:
# compile and train 
model.compile(
    loss=keras.losses.binary_crossentropy,
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [32]:

# adding a callback
class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get("accuracy")) >= 0.99:
            print("\nStopping training as model has hit 99% accuracy")
            self.model.stop_training = True

history = model.fit(
    train_datagen,
    # steps_per_epoch=22,
    epochs = 15,
    validation_data = val_datagen
)

Epoch 1/15
9/9 [==============================] - 122s 14s/step - loss: 0.6881 - accuracy: 0.5482 - val_loss: 0.5817 - val_accuracy: 0.8398
Epoch 2/15
9/9 [==============================] - 139s 17s/step - loss: 0.4084 - accuracy: 0.8520 - val_loss: 1.4043 - val_accuracy: 0.7344
Epoch 3/15
9/9 [==============================] - 127s 14s/step - loss: 0.2245 - accuracy: 0.9085 - val_loss: 1.3288 - val_accuracy: 0.7891
Epoch 4/15
9/9 [==============================] - 101s 11s/step - loss: 0.1453 - accuracy: 0.9367 - val_loss: 1.5273 - val_accuracy: 0.8438
Epoch 5/15
9/9 [==============================] - 193s 22s/step - loss: 0.0874 - accuracy: 0.9679 - val_loss: 1.7234 - val_accuracy: 0.8047
Epoch 6/15
9/9 [==============================] - 79s 9s/step - loss: 0.0688 - accuracy: 0.9747 - val_loss: 2.3715 - val_accuracy: 0.7695
Epoch 7/15
9/9 [==============================] - 198s 23s/step - loss: 0.0731 - accuracy: 0.9708 - val_loss: 2.6854 - val_accuracy: 0.8281
Epoch 8/15
9/9 [======

In [33]:
history.params

{'verbose': 1, 'epochs': 15, 'steps': 9}

In [39]:
path = "data/horses-1414889_640.jpg"

img = load_img(path, target_size=(256, 256))
x = img_to_array(img)
print("Shape:",x.shape)
x /= 255
x = np.expand_dims(x, axis=0)
print("Post expansion:",x.shape)

images = np.vstack([x])
print("Images:", images.shape)
classes = model.predict(images, batch_size=10)
print(classes)

if classes[0]>0.5:
    print(path + " is a human")
else:
    print(path + " is a horse")

Shape: (256, 256, 3)
Post expansion: (1, 256, 256, 3)
Images: (1, 256, 256, 3)
1/1 [==============================] - 0s 263ms/step
[[9.091409e-09]]
data/horses-1414889_640.jpg is a horse
